In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_json('C:/Users/Richard Teller/Desktop/429/Project/Data/train.json')
df.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [3]:
df = df.drop(['id'], axis=1)
df.head()

,cuisine,ingredients
0,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,"[water, vegetable oil, wheat, salt]"
4,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [4]:
from sklearn.cross_validation import train_test_split

X = df['ingredients'].tolist()
y = df['cuisine'].tolist()

# In order to use the text vectorization stuff, we need X_train to be a list, in 
# format of ["ingr1, ingr2", "ingr1, ingr2, ingr3"] where basically each recipe is just a sentence
# with ingredients seperated by a comma.  The following for loop does this, by using a method from String
# that joins the strings in a list, into one string seperated by the specified string, in this case I used
# ', '

new_X = []
for l in X:
    new_X.append(', '.join(l))

X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=.2, random_state=20)
X_train[:10] # X_train is a list, this is how you print the first 10 items

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


['lime, mung beans, sesame oil, peanut butter, carrots, fish sauce, fresh ginger, green onions, cilantro, garlic cloves, monterey jack, honey, hoisin sauce, red pepper, pizza doughs, boiling water, soy sauce, Sriracha, cooked chicken, rice vinegar, oyster sauce',
 'eggs, decorating sugars, softened butter, white sugar, ground cinnamon, milk, salt, confectioners sugar, melted butter, active dry yeast, all-purpose flour, sour cream, warm water, vanilla extract, chopped pecans',
 'olive oil, crushed red pepper, tomatoes, kalamata, dried oregano, sun-dried tomatoes, onions, capers, garlic',
 'butter, spinach leaves, salt, rainbow trout, heavy cream, fennel, freshly ground pepper',
 'chicken stock, veal, all-purpose flour, dried porcini mushrooms, large garlic cloves, onions, marsala wine, fresh thyme, ground allspice, dried thyme, button mushrooms',
 'red chili peppers, lemon grass, cumin seed, fresh ginger, paprika, lime, shallots, coriander seeds, garlic',
 'kale, coriander, green olives

In [5]:
from sklearn.feature_extraction.text import *

# Now that we have our train test split, we can start vectorizing our documents (recipes).

# Tfidf is recommended for short documents (I think).  Our documents are short since they contain around 20
# words or so for a single recipe at max

vectorizer = TfidfVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)  # Creates a matrix that represents the counts of each word
                                                    # where the index represents a word, and the value there
                                                    # is the count of that word in the document.

tfidf_transformer = TfidfTransformer()  # I don't know what this does, I might be doing something redundant here
                                        # All I know is it performs better if you use the transformer
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [6]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train_tfidf, y_train) # Don't ask why I used Multinomial Naive Bayes
                                                    # I was following a tutorial-ish 
                                                    # http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [7]:
X_test_counts = vectorizer.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

y_pred = model.predict(X_test_tfidf)

In [8]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
acc

0.6862350722815839

In [9]:
# This also works, don't need tfidf_transformer
# However, it gets 2% less accuracy

# vectorizer = TfidfVectorizer()
# X_train_counts = vectorizer.fit_transform(X_train)
# model = MultinomialNB().fit(X_train_counts, y_train)
# X_test_counts = vectorizer.transform(X_test)
# y_pred = model.predict(X_test_counts)
# acc = accuracy_score(y_test, y_pred)
# acc